In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import sys

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('../')
os.getcwd()

'/home/lehl/development/QueryOptDeepDive'

In [27]:
import os
import re
import datetime
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def query_file_to_ident(file_name):
    ident = file_name.split('.sql')[0]
    return f"{ident[:-1].zfill(2)}{ident[-1]}"

QUERY_TIMEOUT = 3 * 60 * 1000.0

## Read test queries

In [5]:
"""
TEST_QUERIES = dict()

for folder in os.listdir('bao/queries'):
    if not folder.startswith('stack__'):
        continue
        
    key = folder.split('stack__')[1]
    
    queries = os.listdir(os.path.join('bao', 'queries', folder, 'test'))
    queries = sorted([query_file_to_ident(x) for x in queries])

    TEST_QUERIES[key] = queries
    
for k in sorted(TEST_QUERIES.keys()):
    print(k)
    print(TEST_QUERIES[k])
    print('---' * 30)
"""    

TEST_QUERIES = {
    'base_query_split_1': ['q13__13ad1b8c6bea4fda1892b9fa82cc1ceb9ceb85fc', 'q13__1ddcc8650e17b292bc7344902baffc90c5ae5761', 'q13__935e2051bf80eeafe91aeb6eb719b6b64b9592c2', 'q13__a091adce62743b65c04532e98e8ff3d7e546ea77', 'q13__a3d03772d880754fc4e150d82908757477ae2186', 'q13__add0df9dccb2790c14508e19c9e0deb79fad6ea2', 'q13__d383cd5b4aee7d3f73508e2a1fe5f6d0f7dd42a2', 'q13__d4707be2adfdbc842f42acb1fc16e3a43faf7474', 'q2__q2-001', 'q2__q2-012', 'q2__q2-032', 'q2__q2-035', 'q2__q2-050', 'q2__q2-081', 'q2__q2-094', 'q2__q2-098', 'q7__q7-034', 'q7__q7-036', 'q7__q7-047', 'q7__q7-077', 'q7__q7-082', 'q7__q7-085', 'q7__q7-095', 'q7__q7-099'],
    'leave_one_out_split_1': ['q11__6c5cba419c5b7b02d431aeb5e766d775d812967a', 'q12__547c6bf1994c9b2ba82a7ae32f4b051beabf46fd', 'q13__935e2051bf80eeafe91aeb6eb719b6b64b9592c2', 'q14__5e4835cd72aaa2d7be15b2a5ffa2e66156b3656f', 'q15__543ab3f730e494a69e3d15e59675f491544cb15d', 'q16__b1a96cd48ba297dd93bce73c27b491069ad7449f', 'q1__q1-035', 'q2__q2-032', 'q3__q3-043', 'q4__q4-041', 'q5__q5-041', 'q6__q6-060', 'q7__q7-047', 'q8__q8-046'],
    'random_split_1': ['q11__6c5cba419c5b7b02d431aeb5e766d775d812967a', 'q11__c1ae2a992cde4ea2c4922d852df22043254b4f84', 'q12__55de941e8497cfeeb93d3f8f2d7a18489e0e6c32', 'q14__63c0776f1727638316b966fe748df7cc585a335b', 'q14__74fd1af68d23f0690e3d0fc80bd9b42fa90a7e94', 'q14__97e68ad5c2ced4c182366b3118a1f5f69b423fa6', 'q14__b49361f85785200ed6ec1f2eec357b7598c9e564', 'q15__3e37e62655ceaebc14e79edad518e5710752f51d', 'q15__543ab3f730e494a69e3d15e59675f491544cb15d', 'q15__b8ddf65b0c0c7867a9b560e571d457fec410715c', 'q15__d5546c01928a687eb1f54e9f8eb4e1aff68fc381', 'q16__1e863562a79ca1f7754c759ebab6a2addda0bde8', 'q16__ea9efde510227beb8d624b8c4a6941b9d5e6e637', 'q16__ed2ffeaefcf5ad8bbadc713ccc766541e12080aa', 'q1__q1-031', 'q1__q1-035', 'q4__q4-042', 'q4__q4-064', 'q4__q4-089', 'q5__q5-032', 'q6__q6-060', 'q6__q6-064', 'q7__q7-099'],
}


## Read Postgres results

In [6]:
for f in sorted(os.listdir('bao/logs')):
    if 'stack' in f:
        print(f)

test__bao__stack__base_query_split_1__1.txt
test__bao__stack__base_query_split_1__2.txt
test__bao__stack__leave_one_out_split_1__1.txt
test__bao__stack__leave_one_out_split_1__2.txt
test__bao__stack__random_split_1__1.txt
test__bao__stack__random_split_1__2.txt
test__postgres__stack__0.txt
test__postgres__stack__1.txt
test__postgres__stack__2.txt
test__postgres__stack__3.txt
test__postgres__stack__4.txt
test__postgres__stack__5.txt
test__postgres__stack__Xwarmup_1.txt
test__postgres__stack__Xwarmup_2.txt
test__postgres__stack__cache_up_1.txt
test__postgres__stack__cache_up_10.txt
test__postgres__stack__cache_up_11.txt
test__postgres__stack__cache_up_12.txt
test__postgres__stack__cache_up_13.txt
test__postgres__stack__cache_up_14.txt
test__postgres__stack__cache_up_15.txt
test__postgres__stack__cache_up_16.txt
test__postgres__stack__cache_up_17.txt
test__postgres__stack__cache_up_18.txt
test__postgres__stack__cache_up_19.txt
test__postgres__stack__cache_up_2.txt
test__postgres__stack__c

In [39]:
pg_paths = [
    #'test__postgres__stack__0.txt',
    #'test__postgres__stack__1.txt',
    #'test__postgres__stack__2.txt',
    #'test__postgres__stack__3.txt',
    #'test__postgres__stack__4.txt',
    #'test__postgres__stack__5.txt',
    'test__postgres__stack__cache_up_29.txt',
    'test__postgres__stack__cache_up_30.txt',
    ]

def read_pg_log(file_name, i):
    pg_data = []
    
    path = os.path.join('bao', 'logs', file_name)
    with open(path, 'r') as f:
        lines = f.readlines()
        
    run_type = 'regular'
        
    for line in lines:
        hint, iteration, timestamp, query_path, planning, execution, method = line.split('\n')[0].split(', ')
        
        planning_time = float(planning)
        execution_time = float(execution)
        
        if execution_time >= QUERY_TIMEOUT:
            timed_out = True
            execution_time = 2 * QUERY_TIMEOUT
            
            if planning_time >= QUERY_TIMEOUT:
                planning_time = QUERY_TIMEOUT
        else:
            timed_out = False
        
        tmp = {
            'query_ident': query_file_to_ident(query_path.split('/')[-1]),
            'iteration': int(iteration),
            'planning_time': planning_time,
            'execution_time': execution_time,
            'method': 'PostgreSQL',
            'run_id': i,
            'run_type': run_type,
            'timed_out': timed_out,
            'experiment': None,
        }
        tmp['split'] = None
        pg_data.append(tmp)
        
    df_pg = pd.DataFrame(pg_data)
    df_pg['inference_time'] = 0.0
    df_pg = df_pg[df_pg['iteration'] == 2].reset_index(drop=True)
    df_pg = df_pg.drop(columns=['iteration'])
    df_pg = df_pg[['run_id', 'run_type', 'experiment', 'method', 'query_ident', 'split', 'inference_time', 'planning_time', 'execution_time', 'timed_out']]
    df_pg = df_pg.sort_values(['query_ident']).reset_index(drop=True)
    return df_pg
        
all_pg_df = []
cmp_data = dict()
for i, p in enumerate(pg_paths):
    df_tmp = read_pg_log(p, i)
    print(f"[{i}] Planning Time: {df_tmp.sum(numeric_only=True)['planning_time']:.2f}\tExecution Time: {df_tmp.sum(numeric_only=True)['execution_time']:.2f}\t{p}")
    cmp_data[i] = df_tmp['execution_time'].tolist()
    #display(df_tmp)
    all_pg_df.append(df_tmp)
    
df_pg_all = pd.concat(all_pg_df)
display(df_pg_all[df_pg_all['query_ident'] == 'q13__935e2051bf80eeafe91aeb6eb719b6b64b9592c2'])

[0] Planning Time: 5765.54	Execution Time: 81604.01	test__postgres__stack__cache_up_29.txt
[1] Planning Time: 5739.20	Execution Time: 76632.24	test__postgres__stack__cache_up_30.txt


,run_id,run_type,experiment,method,query_ident,split,inference_time,planning_time,execution_time,timed_out
18,0,regular,None,PostgreSQL,q13__935e2051bf80eeafe91aeb6eb719b6b64b9592c2,None,0.0,73.275,4681.123,False
18,1,regular,None,PostgreSQL,q13__935e2051bf80eeafe91aeb6eb719b6b64b9592c2,None,0.0,73.285,4343.537,False


## Read Bao results (based on Bao codebase)

In [8]:
bao_data = []

for path in sorted(os.listdir(os.path.join('bao', 'logs'))):
    if 'bao' not in path:
        continue
        
    if 'stack' not in path:
        continue
        
    if 'test' not in path:
        continue
        
    if 'with_hint' in path:
        continue
    
    with open(os.path.join('bao', 'logs', path), 'r') as f:
        lines = f.readlines()

    num = path.split('.txt')[0][-3:]
    run_id = (int(num[-1]) - 1) if num[:2] == '__' else 0

    # Pattern to find the train test split 
    pattern = re.compile('([a-z]+\_)+split\_[1-9]')
    experiment = pattern.search(path).group()
    
    for line in lines:
        hint, iteration, timestamp, query_path, planning, execution, method = line.split('\n')[0].split(', ')
    
        planning_time = float(planning)
        execution_time = float(execution)
        
        if execution_time >= QUERY_TIMEOUT:
            timed_out = True
            execution_time = 2 * QUERY_TIMEOUT
            
            if planning_time >= QUERY_TIMEOUT:
                planning_time = QUERY_TIMEOUT
        else:
            timed_out = False
    
        tmp = {
            'query_ident': query_file_to_ident(query_path.split('/')[-1]),
            'iteration': int(iteration),
            'planning_time': planning_time,
            'execution_time': execution_time,
            'method': 'Bao',
            'experiment': experiment,
            'timed_out': timed_out,
            'run_id': run_id,
            'run_type': 'regular',
            'bao_hint': hint,
        }
        experiment_key = tmp['experiment']
        tmp['split'] = 'test' if tmp['query_ident'] in TEST_QUERIES[experiment_key] else 'train'
        bao_data.append(tmp)
            
df_bao = pd.DataFrame(bao_data)
df_bao['inference_time'] = 0.0
df_bao = df_bao[df_bao['iteration'] == 2].reset_index(drop=True)
df_bao = df_bao.drop(columns=['iteration'])
#df_bao = df_bao[['run_id', 'run_type', 'experiment', 'method', 'query_ident', 'split', 'inference_time', 'planning_time', 'execution_time', 'timed_out']]

df_bao[df_bao['query_ident']=='q1__q1-009']

,query_ident,planning_time,execution_time,method,experiment,timed_out,run_id,run_type,bao_hint,split,inference_time
48,q1__q1-009,407.821,14.315,Bao,base_query_split_1,False,0,regular,(no hint),train,0.0
160,q1__q1-009,412.709,14.340,Bao,base_query_split_1,False,1,regular,SET enable_mergejoin TO off; SET enable_indexs...,train,0.0
272,q1__q1-009,409.433,14.235,Bao,leave_one_out_split_1,False,0,regular,(no hint),train,0.0
384,q1__q1-009,397.037,13.792,Bao,leave_one_out_split_1,False,1,regular,(no hint),train,0.0
496,q1__q1-009,416.095,14.552,Bao,random_split_1,False,0,regular,(no hint),train,0.0
608,q1__q1-009,410.015,14.183,Bao,random_split_1,False,1,regular,(no hint),train,0.0


## Read Neo + Balsa results (based on Balsa codebase)

In [12]:
file_paths = {
    'neo': {
        0: {
            'base_query_split_1': 'balsa/logs/2023_12_13__104219__Neo_STACK_BaseQuerySplit1__plan_and_execute.txt',
            'leave_one_out_split_1': 'balsa/logs/2023_12_12__090523__Neo_STACK_LeaveOneOutSplit1__plan_and_execute.txt',
            'random_split_1': 'balsa/logs/2023_12_11__142407__Neo_STACK_RandomSplit1__plan_and_execute.txt',
        },
        1: {
            'base_query_split_1': 'balsa/logs/2023_12_30__032738__Neo_STACK_BaseQuerySplit1__plan_and_execute.txt',
            'leave_one_out_split_1': 'balsa/logs/2023_12_30__055222__Neo_STACK_LeaveOneOutSplit1__plan_and_execute.txt',
            'random_split_1': 'balsa/logs/2023_12_30__004111__Neo_STACK_RandomSplit1__plan_and_execute.txt',
        },
    },
    'balsa': {
        0: {
            'base_query_split_1': 'balsa/logs/2023_12_12__090645__Balsa_STACKBaseQuerySplit1__plan_and_execute.txt',
            'leave_one_out_split_1': 'balsa/logs/2023_12_12__113836__Balsa_STACKLeaveOneOutSplit1__plan_and_execute.txt',
            'random_split_1': 'balsa/logs/2023_12_08__101857__Balsa_STACKRandomSplit1__plan_and_execute.txt',
        },
        1: {
            'base_query_split_1': 'balsa/logs/2023_12_30__020412__Balsa_STACKBaseQuerySplit1__plan_and_execute.txt',
            'leave_one_out_split_1': 'balsa/logs/2023_12_30__042929__Balsa_STACKLeaveOneOutSplit1__plan_and_execute.txt',
            'random_split_1': 'balsa/logs/2023_12_30__004113__Balsa_STACKRandomSplit1__plan_and_execute.txt',
        },
    }
}
file_paths

{'neo': {0: {'base_query_split_1': 'balsa/logs/2023_12_13__104219__Neo_STACK_BaseQuerySplit1__plan_and_execute.txt',
   'leave_one_out_split_1': 'balsa/logs/2023_12_12__090523__Neo_STACK_LeaveOneOutSplit1__plan_and_execute.txt',
   'random_split_1': 'balsa/logs/2023_12_11__142407__Neo_STACK_RandomSplit1__plan_and_execute.txt'},
  1: {'base_query_split_1': 'balsa/logs/2023_12_30__032738__Neo_STACK_BaseQuerySplit1__plan_and_execute.txt',
   'leave_one_out_split_1': 'balsa/logs/2023_12_30__055222__Neo_STACK_LeaveOneOutSplit1__plan_and_execute.txt',
   'random_split_1': 'balsa/logs/2023_12_30__004111__Neo_STACK_RandomSplit1__plan_and_execute.txt'}},
 'balsa': {0: {'base_query_split_1': 'balsa/logs/2023_12_12__090645__Balsa_STACKBaseQuerySplit1__plan_and_execute.txt',
   'leave_one_out_split_1': 'balsa/logs/2023_12_12__113836__Balsa_STACKLeaveOneOutSplit1__plan_and_execute.txt',
   'random_split_1': 'balsa/logs/2023_12_08__101857__Balsa_STACKRandomSplit1__plan_and_execute.txt'},
  1: {'base

In [13]:
df_neo_balsa = None
for method in file_paths.keys():
    for run_id in file_paths[method].keys():
        
        method_paths = file_paths[method][run_id]
        for experiment, file_path in method_paths.items():

            df_tmp = pd.read_csv(file_path, header=None, sep=';')
            df_tmp.columns = ['query_ident', 'inference_time', 'planning_time', 'execution_time']
            df_tmp['query_ident'] = df_tmp['query_ident'].apply(lambda x: query_file_to_ident(x))
            df_tmp['method'] = 'Neo' if method == 'neo' else 'Balsa'
            df_tmp['experiment'] = experiment

            df_tmp['timed_out'] = df_tmp['execution_time'].apply(lambda x: True if x < 0 else False)
            df_tmp['run_id'] = int(run_id)
            df_tmp['run_type'] = 'regular'

            df_tmp.loc[df_tmp['planning_time'] == -1, 'planning_time'] = 0.0
            df_tmp.loc[df_tmp['execution_time'] == -1, 'execution_time'] = QUERY_TIMEOUT
            df_tmp['inference_time'] *= 1000.0

            #if len(df_tmp[df_tmp['timed_out']]) > 0:
            #    display(df_tmp[df_tmp['timed_out']])
            
            df_tmp['split'] = df_tmp['query_ident'].apply(lambda x: 'test' if x in TEST_QUERIES[experiment] else 'train')

            if df_neo_balsa is None:
                df_neo_balsa = df_tmp
            else:
                df_neo_balsa = pd.concat([df_neo_balsa, df_tmp]).reset_index(drop=True)
df_neo_balsa = df_neo_balsa[['run_id', 'run_type', 'experiment', 'method', 'query_ident', 'split', 'inference_time', 'planning_time', 'execution_time', 'timed_out']]
            
display(df_neo_balsa)

,run_id,run_type,experiment,method,query_ident,split,inference_time,planning_time,execution_time,timed_out
0,0,regular,base_query_split_1,Neo,q11__0ea8bacde0e13a4314466435cf49c8e685b39fb1,train,546.0,0.836,98.748,False
1,0,regular,base_query_split_1,Neo,q11__33e1caf220e5bea2e592c82eede1c0427e2c2570,train,97.6,0.793,78.660,False
2,0,regular,base_query_split_1,Neo,q11__6c5cba419c5b7b02d431aeb5e766d775d812967a,train,92.2,2.294,104.509,False
3,0,regular,base_query_split_1,Neo,q11__87c4bd0930b02a3361ac2e86c453db1fec60dc6b,train,57.2,2.579,372.678,False
4,0,regular,base_query_split_1,Neo,q11__9389f58853715321e2a60ad743f99fc365f040cb,train,57.0,0.874,45.813,False
...,...,...,...,...,...,...,...,...,...,...
1339,1,regular,random_split_1,Balsa,q4__q4-089,test,108.3,2.977,10572.707,False
1340,1,regular,random_split_1,Balsa,q5__q5-032,test,153.2,7.626,2225.968,False
1341,1,regular,random_split_1,Balsa,q6__q6-060,test,82.6,0.000,180000.000,True
1342,1,regular,random_split_1,Balsa,q6__q6-064,test,77.4,2.846,10627.374,False


## Read LEON results

In [33]:
leon_files = [
    'log__stack__base_query_split_1.csv',
    'log__stack__leave_one_out_split_1.csv',
    'log__stack__random_split_1.csv',
    'log__stack__random_split_1__2.csv',
    'log__stack__base_query_split_1__2.csv',
    'log__stack__leave_one_out_split_1__2.csv'
]


dfs = []
for file_name in leon_files:
    file_path = os.path.join('leon', file_name)
    
    df_tmp = pd.read_csv(file_path, sep=';')
    df_tmp['query_ident'] = df_tmp['query_ident'].apply(query_file_to_ident)
    experiment = file_name.split('__')[2].split('.csv')[0]
    
    df_tmp['experiment'] = experiment
    df_tmp['run_type'] = 'regular'
    df_tmp['run_id'] = 1 if '__2' in file_name else 0
    df_tmp['method'] = 'LEON'
    
    df_tmp['timed_out'] = False
    df_tmp.loc[df_tmp[df_tmp['execution_time'] > QUERY_TIMEOUT].index, 'timed_out'] = True
    df_tmp.loc[df_tmp[df_tmp['execution_time'] > QUERY_TIMEOUT].index, 'execution_time'] = QUERY_TIMEOUT
    df_tmp['inference_time'] *= 1000.0
    
    dfs.append(df_tmp)

if len(dfs) > 0:
    df_leon = pd.concat(dfs).sort_values(['experiment', 'query_ident', 'run_id']).reset_index(drop=True)
    display(df_leon)
else:
    None

,query_ident,split,inference_time,planning_time,execution_time,experiment,run_type,run_id,method,timed_out
0,q11__0ea8bacde0e13a4314466435cf49c8e685b39fb1,train,1.201630,3.531,115.959,base_query_split_1,regular,0,LEON,False
1,q11__0ea8bacde0e13a4314466435cf49c8e685b39fb1,train,1.275301,3.385,103.561,base_query_split_1,regular,1,LEON,False
2,q11__33e1caf220e5bea2e592c82eede1c0427e2c2570,train,1.416445,3.496,85.909,base_query_split_1,regular,0,LEON,False
3,q11__33e1caf220e5bea2e592c82eede1c0427e2c2570,train,1.373291,1.320,48.285,base_query_split_1,regular,1,LEON,False
4,q11__6c5cba419c5b7b02d431aeb5e766d775d812967a,train,1.219749,3.522,177.193,base_query_split_1,regular,0,LEON,False
...,...,...,...,...,...,...,...,...,...,...
667,q8__q8-074,train,380090.330839,64.840,3045.404,random_split_1,regular,1,LEON,False
668,q8__q8-076,train,382942.129850,0.000,180000.000,random_split_1,regular,0,LEON,True
669,q8__q8-076,train,375050.894260,64.044,805.825,random_split_1,regular,1,LEON,False
670,q8__q8-096,train,381238.924742,62.394,618.151,random_split_1,regular,0,LEON,False


## Read HybridQO results

In [30]:
hybridqo_paths = []
for f in sorted(os.listdir('hybrid_qo/logs/wandb_export')):
    if 'STACK' in f:
        hybridqo_paths.append(f)
        
        
# Prepare query identifiers from workloads
hybridqo_query_idents = {
    'random_split_1': [],
    'base_query_split_1': [],
    'leave_one_out_split_1': [],
}

for workload in hybridqo_query_idents.keys():
    workload_path = f'hybrid_qo/workload/STACK__{workload}__'
    
    train_path = workload_path + 'train.json'
    test_path = workload_path + 'test.json'
    
    with open(train_path) as f:
        tmp_workload = json.load(f)
    hybridqo_query_idents[workload].extend([line[-2].replace('.sql','') for line in tmp_workload])
    
    with open(test_path) as f:
        tmp_workload = json.load(f)
    hybridqo_query_idents[workload].extend([line[-2].replace('.sql','') for line in tmp_workload])

    
dfs = []
for file_name in hybridqo_paths:
    file_path = os.path.join('hybrid_qo', 'logs', 'wandb_export', file_name)
    
    df_tmp = pd.read_csv(file_path)
    df_tmp.columns

    known_column_names = ['hinter_latency', 'test_query', 'epoch', 'mcts_time', 'hinter_plan_time', 'MPHE_time']

    new_columns = []
    select_columns = []
    for c in df_tmp.columns:
        for known_col in known_column_names:
            found = False
            if c.endswith(known_col):
                new_columns.append(known_col)
                select_columns.append(known_col)
                found = True
                break
        if not found:
            new_columns.append(c)

    df_tmp.columns = new_columns
    df_tmp = df_tmp[['Step'] + select_columns]

    df_tmp = df_tmp[df_tmp['epoch'] == df_tmp['epoch'].max()]
    df_tmp['execution_time'] = 1000.0 * df_tmp['hinter_latency']
    df_tmp['planning_time'] = 1000.0 * df_tmp['hinter_plan_time']
    df_tmp['inference_time'] = 1000.0 * (df_tmp['mcts_time'] + df_tmp['MPHE_time'])

    df_tmp['timed_out'] = df_tmp['execution_time'] >= QUERY_TIMEOUT

    file_name = file_name.split('.csv')[0]
    method, workload, experiment, run_id = file_name.split('__')
    
    df_tmp['run_id'] = int(run_id)
    df_tmp['run_type'] = 'regular'
    df_tmp['experiment'] = experiment
    df_tmp['method'] = 'HybridQO'
    df_tmp['query_ident'] = hybridqo_query_idents[experiment]
    df_tmp['split'] = df_tmp['test_query'].apply(lambda x: 'train' if x == 0 else 'test')

    df_tmp = df_tmp[['run_id', 'run_type', 'experiment', 'method', 'query_ident', 'split', 'inference_time', 'planning_time', 'execution_time', 'timed_out']]
    dfs.append(df_tmp)
    

if len(dfs) > 0:
    df_hybridqo = pd.concat(dfs).sort_values(['experiment', 'query_ident', 'run_id']).reset_index(drop=True)
    display(df_hybridqo)
else:
    None


,run_id,run_type,experiment,method,query_ident,split,inference_time,planning_time,execution_time,timed_out
0,0,regular,base_query_split_1,HybridQO,q11__0ea8bacde0e13a4314466435cf49c8e685b39fb1,train,38.178205,30.856,440.162,False
1,1,regular,base_query_split_1,HybridQO,q11__0ea8bacde0e13a4314466435cf49c8e685b39fb1,train,24.524212,9.965,35.803,False
2,0,regular,base_query_split_1,HybridQO,q11__33e1caf220e5bea2e592c82eede1c0427e2c2570,train,21.019936,1.309,48.887,False
3,1,regular,base_query_split_1,HybridQO,q11__33e1caf220e5bea2e592c82eede1c0427e2c2570,train,37.935257,32.280,131.146,False
4,0,regular,base_query_split_1,HybridQO,q11__6c5cba419c5b7b02d431aeb5e766d775d812967a,train,20.853043,352.783,18.433,False
...,...,...,...,...,...,...,...,...,...,...
667,1,regular,random_split_1,HybridQO,q8__q8-074,train,21.888018,405.159,18.805,False
668,0,regular,random_split_1,HybridQO,q8__q8-076,train,22.441387,9.865,42.001,False
669,1,regular,random_split_1,HybridQO,q8__q8-076,train,43.582916,98.375,861.274,False
670,0,regular,random_split_1,HybridQO,q8__q8-096,train,36.224604,66.051,621.058,False


## Combine all results into a single dataframe

In [34]:
#df = pd.concat([df_pg_all, df_bao, df_neo_balsa, df_hybridqo]).sort_values(['experiment', 'method', 'query_ident']).reset_index(drop=True)
df = pd.concat([df_pg_all, df_bao, df_neo_balsa, df_leon, df_hybridqo]).sort_values(['experiment', 'method', 'query_ident']).reset_index(drop=True)
df['total_time'] = df['inference_time'] + df['planning_time'] + df['execution_time']
df = df[['run_id', 'run_type', 'experiment', 'method', 'query_ident', 'split', 'inference_time', 'planning_time', 'execution_time', 'total_time', 'timed_out']]
df

,run_id,run_type,experiment,method,query_ident,split,inference_time,planning_time,execution_time,total_time,timed_out
0,0,regular,base_query_split_1,Balsa,q11__0ea8bacde0e13a4314466435cf49c8e685b39fb1,train,87.9,1.083,271.606,360.589,False
1,1,regular,base_query_split_1,Balsa,q11__0ea8bacde0e13a4314466435cf49c8e685b39fb1,train,992.5,1.082,269.814,1263.396,False
2,0,regular,base_query_split_1,Balsa,q11__33e1caf220e5bea2e592c82eede1c0427e2c2570,train,115.5,1.728,268.531,385.759,False
3,1,regular,base_query_split_1,Balsa,q11__33e1caf220e5bea2e592c82eede1c0427e2c2570,train,93.8,1.971,775.176,870.947,False
4,0,regular,base_query_split_1,Balsa,q11__6c5cba419c5b7b02d431aeb5e766d775d812967a,train,128.6,1.241,293.943,423.784,False
...,...,...,...,...,...,...,...,...,...,...,...
3579,1,regular,None,PostgreSQL,q8__q8-074,None,0.0,335.158,25.955,361.113,False
3580,0,regular,None,PostgreSQL,q8__q8-076,None,0.0,354.519,19.981,374.500,False
3581,1,regular,None,PostgreSQL,q8__q8-076,None,0.0,348.760,19.255,368.015,False
3582,0,regular,None,PostgreSQL,q8__q8-096,None,0.0,347.515,19.239,366.754,False


In [37]:
df.groupby(['run_type', 'experiment', 'method', 'timed_out'], dropna=False).count()

run_id  query_ident  \
run_type experiment            method     timed_out                        
regular  base_query_split_1    Balsa      False         161          161   
                                          True           63           63   
                               Bao        False         224          224   
                               HybridQO   False         224          224   
                               LEON       False         208          208   
                                          True           16           16   
                               Neo        False         170          170   
                                          True           54           54   
         leave_one_out_split_1 Balsa      False         221          221   
                                          True            3            3   
                               Bao        False         224          224   
                               HybridQO   False         224          224   
                               LEON       False         217          217   
                                          True            7            7   
                               Neo        False         190          190   
                                          True           34           34   
         random_split_1        Balsa      False         208          208   
                                          True           16           16   
                               Bao        False         224          224   
                               HybridQO   False         222          222   
                                          True            2            2   
                               LEON       False         193          193   
                                          True           31           31   
                               Neo        False         145          145   
                                          True           79           79   
         NaN                   PostgreSQL False         224          224   

                                                     split  inference_time  \
run_type experiment            method     timed_out                          
regular  base_query_split_1    Balsa      False        161             161   
                                          True          63              63   
                               Bao        False        224             224   
                               HybridQO   False        224             224   
                               LEON       False        208             208   
                                          True          16              16   
                               Neo        False        170             170   
                                          True          54              54   
         leave_one_out_split_1 Balsa      False        221             221   
                                          True           3               3   
                               Bao        False        224             224   
                               HybridQO   False        224             224   
                               LEON       False        217             217   
                                          True           7               7   
                               Neo        False        190             190   
                                          True          34              34   
         random_split_1        Balsa      False        208             208   
                                          True          16              16   
                               Bao        False        224             224   
                               HybridQO   False        222             222   
                                          True           2               2   
                               LEON       False        193             193   
                                          True      

In [40]:
new_file_name = 'experiment_logs/' + datetime.datetime.now().strftime('%Y%m%d') + '__STACK__combined.csv'
df.to_csv(new_file_name, index=False)

print(f"Saved combined results to {new_file_name}")

Saved combined results to experiment_logs/20240103__STACK__combined.csv
